✋ The goal is to build a machine learning model that can automatically assess the relevance of Arabic text documents to a specific topic, assigning a score between 0 and 10.


*  **Steps: ✈**


1.   Data Collection and Preparation:

Choose a Topic and Websites: Select a relevant topic (e.g., "technology," "LLMs") and find Arabic websites covering that topic.

Web Scraping: Use libraries like Beautiful Soup (requests and BeautifulSoup in Python) to extract text content from articles on those websites.

Scoring: Create a dataset by assigning a relevance score (0-10) to each scraped text article using Embedding and TF-IDF techniques.

2. Arabic Text Preprocessing:

  * Segmentation: Split the Arabic text into individual words using a tool like
  Farasa (FarasaSegmenter).

  * Stop Word Removal: Eliminate common Arabic words (like prepositions and conjunctions) that don't carry much meaning using nltk.corpus.stopwords.

  * Stemming: Reduce words to their root form using either light stemming (ArabicLightStemmer from tashaphyne.stemming) or the ISRI Stemmer (ISRIStemmer from nltk.stem.isri).

  * Lemmatization (optional): If possible, convert words to their dictionary form (lemma) using an Arabic lemmatizer. This can improve accuracy but might be more complex.

3. Data Splitting:

  * Divide the dataset into training, validation, and test sets using train_test_split from sklearn.model_selection.

4. Text Vectorization (Numerical Representation):
  * Tokenization: Convert words into numerical representations using Tokenizer from tensorflow.keras.preprocessing.text.
  * Padding: Ensure all text sequences have the same length using pad_sequences from tensorflow.keras.preprocessing.sequence.

5. Building and Training RNN Models:

   Model Architectures: Experiment with different Recurrent Neural Network (RNN) architectures using Keras:
  * Simple RNN (SimpleRNN)
  * Bidirectional RNN (Bidirectional(LSTM) or Bidirectional(GRU))
  * GRU (GRU)
  * LSTM (LSTM)

  Model Compilation: Compile the models using an appropriate loss function (like Mean Squared Error for regression), an optimizer (e.g., adam), and evaluation metrics (e.g., Mean Absolute Error).

  Model Training: Train the models on prepared data and validate their performance using the validation set.

6. Model Evaluation:

  Evaluation Metrics: Evaluate the trained models on the test set using metrics such as:

  * Mean Squared Error (MSE)
  * R-squared (R²)

  ▶▶▶
  


In [211]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.isri import ISRIStemmer
from qalsadi.lemmatizer import Lemmatizer
import nltk
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import torch
from transformers import BertTokenizer, BertModel
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Bidirectional, Dense, SimpleRNN, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [170]:
df = pd.read_csv('/content/drive/Othercomputers/My Laptop/ArabicData/scraped_data.csv' , header=None)
df.columns = ['Text']
df.head()

,Text
0,نماذج اللغة الكبيرة (LLM) هي نماذج
1,كبيرة جدًا مدرَّبة مسبقًا على كميات هائلة من ...
2,التي تتكون من وحدتَي تشفير وفك تشفير مع قدرات...
3,إن نماذج اللغة الكبيرة القائمة على المحوِّل قا...
4,على عكس الشبكات العصبونية المتكررة (RNN) التي ...


# **CLEAN DATA ⌛**


In [171]:
# nltk.download('stopwords')
df['Text_cleaned'] = df['Text'].str.replace(r'[^\u0600-\u06FF\s]+' ,'', regex=True)
stop_words = set(stopwords.words('arabic'))
df['Text_cleaned'] = df['Text_cleaned'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
df.head()

,Text,Text_cleaned
0,نماذج اللغة الكبيرة (LLM) هي نماذج,نماذج اللغة الكبيرة نماذج
1,كبيرة جدًا مدرَّبة مسبقًا على كميات هائلة من ...,كبيرة جدًا مدرَّبة مسبقًا كميات هائلة البيانات...
2,التي تتكون من وحدتَي تشفير وفك تشفير مع قدرات...,تتكون وحدتَي تشفير وفك تشفير قدرات الانتباه ال...
3,إن نماذج اللغة الكبيرة القائمة على المحوِّل قا...,نماذج اللغة الكبيرة القائمة المحوِّل قادرة الت...
4,على عكس الشبكات العصبونية المتكررة (RNN) التي ...,عكس الشبكات العصبونية المتكررة ظهرت السابق وتع...


# **ADD SCORE TO DATA [ TF-IDF vs Embedding ] ✅**

In [172]:
topic = 'نماذج اللغة الكبيرة'

# all = [topic] + [''.join(text) for i , text in df['lemm Text'].items() if text is not None]
all = [topic] + df['Text_cleaned'].tolist()
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(all)
# print(tfidf_matrix[0])
# print(tfidf_matrix[1:])
cosine_sim = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1:]).flatten()
for i , score in enumerate(cosine_sim):
  print(i , score)

0 0.943183346140317
1 0.0
2 0.0
3 0.1677925027550678
4 0.1521500371132732
5 0.050758226894803006
6 0.0
7 0.28415846161987623
8 0.20978231872283482
9 0.0
10 0.22467981193620282
11 0.08108417619814448
12 0.04014855229880923
13 0.1300777924361294
14 0.0
15 0.15200955033066044
16 0.061770484804173874
17 0.0976379832719534
18 0.0
19 0.0
20 0.029879171435515706
21 0.0
22 0.11915737865275773
23 0.14194659823998637
24 0.2809772702259517
25 0.09653711607156323
26 0.19457772978443577
27 0.15919478961574304
28 0.1242908461187018
29 0.19441049739300123
30 0.0
31 0.285016566435093
32 0.06113625470044178
33 0.24520342069270706
34 0.0
35 0.0
36 0.0
37 0.5880242817936283
38 0.0
39 0.0
40 0.0
41 0.0
42 0.0
43 0.0
44 0.310675342644293
45 0.09766576468005422
46 0.07275629130391528
47 0.05733081765724747
48 0.0
49 0.0
50 0.0
51 0.0
52 0.0
53 0.0
54 0.055805674118539234
55 0.0
56 0.0
57 0.0
58 0.0
59 0.031141746316102805
60 0.10277198010604235
61 0.0
62 0.0
63 0.18491868436487988
64 0.12882128485058728
65 

In [173]:
topic = 'نماذج اللغة الكبيرة'

texts = df['Text_cleaned'].tolist()

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_embedding(text , tokenizer):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

topic_embedding = get_embedding(topic , tokenizer)
text_embeddings = np.array([get_embedding(text , tokenizer) for text in texts]).squeeze(1)

relevance_scores = cosine_similarity(topic_embedding, text_embeddings).flatten()
scores = []
for i, score in enumerate(relevance_scores):
    print(f"Text {i+1} relevance score: {score}")
    scores.append(f"{score*10:.1f}")
df['relevance_score'] = scores
df['relevance_score'] = pd.to_numeric(df['relevance_score'])
df.head()

Text 1 relevance score: 0.9755904674530029
Text 2 relevance score: 0.942656397819519
Text 3 relevance score: 0.9054958820343018
Text 4 relevance score: 0.9297724962234497
Text 5 relevance score: 0.906392514705658
Text 6 relevance score: 0.9108273983001709
Text 7 relevance score: 0.9053435921669006
Text 8 relevance score: 0.9143202304840088
Text 9 relevance score: 0.9224518537521362
Text 10 relevance score: 0.948388397693634
Text 11 relevance score: 0.9421145915985107
Text 12 relevance score: 0.8896218538284302
Text 13 relevance score: 0.9028751254081726
Text 14 relevance score: 0.9673269987106323
Text 15 relevance score: 0.917568564414978
Text 16 relevance score: 0.9329664707183838
Text 17 relevance score: 0.9052997827529907
Text 18 relevance score: 0.9474372267723083
Text 19 relevance score: 0.9091063737869263
Text 20 relevance score: 0.9106945991516113
Text 21 relevance score: 0.8983892202377319
Text 22 relevance score: 0.905248761177063
Text 23 relevance score: 0.9161394834518433
Te

,Text,Text_cleaned,relevance_score
0,نماذج اللغة الكبيرة (LLM) هي نماذج,نماذج اللغة الكبيرة نماذج,9.8
1,كبيرة جدًا مدرَّبة مسبقًا على كميات هائلة من ...,كبيرة جدًا مدرَّبة مسبقًا كميات هائلة البيانات...,9.4
2,التي تتكون من وحدتَي تشفير وفك تشفير مع قدرات...,تتكون وحدتَي تشفير وفك تشفير قدرات الانتباه ال...,9.1
3,إن نماذج اللغة الكبيرة القائمة على المحوِّل قا...,نماذج اللغة الكبيرة القائمة المحوِّل قادرة الت...,9.3
4,على عكس الشبكات العصبونية المتكررة (RNN) التي ...,عكس الشبكات العصبونية المتكررة ظهرت السابق وتع...,9.1


# **PRE-PROCESS DATA**

In [174]:
df['Text tokens'] = df['Text_cleaned'].apply(word_tokenize)
df.head()

,Text,Text_cleaned,relevance_score,Text tokens
0,نماذج اللغة الكبيرة (LLM) هي نماذج,نماذج اللغة الكبيرة نماذج,9.8,"[نماذج, اللغة, الكبيرة, نماذج]"
1,كبيرة جدًا مدرَّبة مسبقًا على كميات هائلة من ...,كبيرة جدًا مدرَّبة مسبقًا كميات هائلة البيانات...,9.4,"[كبيرة, جدًا, مدرَّبة, مسبقًا, كميات, هائلة, ا..."
2,التي تتكون من وحدتَي تشفير وفك تشفير مع قدرات...,تتكون وحدتَي تشفير وفك تشفير قدرات الانتباه ال...,9.1,"[تتكون, وحدتَي, تشفير, وفك, تشفير, قدرات, الان..."
3,إن نماذج اللغة الكبيرة القائمة على المحوِّل قا...,نماذج اللغة الكبيرة القائمة المحوِّل قادرة الت...,9.3,"[نماذج, اللغة, الكبيرة, القائمة, المحوِّل, قاد..."
4,على عكس الشبكات العصبونية المتكررة (RNN) التي ...,عكس الشبكات العصبونية المتكررة ظهرت السابق وتع...,9.1,"[عكس, الشبكات, العصبونية, المتكررة, ظهرت, السا..."


In [175]:
stemmer = ISRIStemmer()
stemm  = []
for i in range(len(df['Text tokens'])):
  stemm.append([stemmer.stem(word) for word in df['Text tokens'][i]])
df['stem Text'] = stemm
df.head(2)

,Text,Text_cleaned,relevance_score,Text tokens,stem Text
0,نماذج اللغة الكبيرة (LLM) هي نماذج,نماذج اللغة الكبيرة نماذج,9.8,"[نماذج, اللغة, الكبيرة, نماذج]","[اذج, لغة, كبر, اذج]"
1,كبيرة جدًا مدرَّبة مسبقًا على كميات هائلة من ...,كبيرة جدًا مدرَّبة مسبقًا كميات هائلة البيانات...,9.4,"[كبيرة, جدًا, مدرَّبة, مسبقًا, كميات, هائلة, ا...","[كبر, جدا, درب, سبق, كمي, هئل, بين, حول, سسي, ..."


In [176]:
lemmatiser = Lemmatizer()
lemm  = []
for i in range(len(df['Text tokens'])):
  lemm.append([lemmatiser.lemmatize(word) for word in df['Text tokens'][i]])
df['lemm Text'] = lemm
df.head(2)

Exception ignored in: <function ArabicDictionary.__del__ at 0x781ed4f57a30>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/arramooz/arabicdictionary.py", line 109, in __del__
    self.db_connect.close()
sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 132078276694016 and this is thread id 132077584361024.
Exception ignored in: <function ArabicDictionary.__del__ at 0x781ed4f57a30>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/arramooz/arabicdictionary.py", line 109, in __del__
    self.db_connect.close()
sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 132078276694016 and this is thread id 132077584361024.


,Text,Text_cleaned,relevance_score,Text tokens,stem Text,lemm Text
0,نماذج اللغة الكبيرة (LLM) هي نماذج,نماذج اللغة الكبيرة نماذج,9.8,"[نماذج, اللغة, الكبيرة, نماذج]","[اذج, لغة, كبر, اذج]","[نماذج, لغة, كبير, نماذج]"
1,كبيرة جدًا مدرَّبة مسبقًا على كميات هائلة من ...,كبيرة جدًا مدرَّبة مسبقًا كميات هائلة البيانات...,9.4,"[كبيرة, جدًا, مدرَّبة, مسبقًا, كميات, هائلة, ا...","[كبر, جدا, درب, سبق, كمي, هئل, بين, حول, سسي, ...","[كبير, جد, مدرب, مسبق, كم, هائل, بيان, محول, أ..."


In [177]:
df.to_csv('/content/drive/Othercomputers/My Laptop/ArabicData/scraped_data_with_score.csv' , index=False)

In [178]:
df

,Text,Text_cleaned,relevance_score,Text tokens,stem Text,lemm Text
0,نماذج اللغة الكبيرة (LLM) هي نماذج,نماذج اللغة الكبيرة نماذج,9.8,"[نماذج, اللغة, الكبيرة, نماذج]","[اذج, لغة, كبر, اذج]","[نماذج, لغة, كبير, نماذج]"
1,كبيرة جدًا مدرَّبة مسبقًا على كميات هائلة من ...,كبيرة جدًا مدرَّبة مسبقًا كميات هائلة البيانات...,9.4,"[كبيرة, جدًا, مدرَّبة, مسبقًا, كميات, هائلة, ا...","[كبر, جدا, درب, سبق, كمي, هئل, بين, حول, سسي, ...","[كبير, جد, مدرب, مسبق, كم, هائل, بيان, محول, أ..."
2,التي تتكون من وحدتَي تشفير وفك تشفير مع قدرات...,تتكون وحدتَي تشفير وفك تشفير قدرات الانتباه ال...,9.1,"[تتكون, وحدتَي, تشفير, وفك, تشفير, قدرات, الان...","[تتك, وحد, شفر, وفك, شفر, قدر, نبه, ذتي, خرج, ...","[تك, وحد, تشفير, فك, تشفير, قدر, انتباه, ذاتي,..."
3,إن نماذج اللغة الكبيرة القائمة على المحوِّل قا...,نماذج اللغة الكبيرة القائمة المحوِّل قادرة الت...,9.3,"[نماذج, اللغة, الكبيرة, القائمة, المحوِّل, قاد...","[اذج, لغة, كبر, قئم, حول, قدر, درب, بدن, شرف،,...","[نماذج, لغة, كبير, قائم, محول, قادر, تدريب, بد..."
4,على عكس الشبكات العصبونية المتكررة (RNN) التي ...,عكس الشبكات العصبونية المتكررة ظهرت السابق وتع...,9.1,"[عكس, الشبكات, العصبونية, المتكررة, ظهرت, السا...","[عكس, شبك, عصبو, تكرر, ظهر, سبق, علج, دخل, بع،...","[عكس, شبكة, العصبونية, متكرر, ظهر, سابق, تعالج..."
...,...,...,...,...,...,...
101,بالإشارة إلى الخطط للسماح للمستخدمين بتخصيص س...,بالإشارة الخطط للسماح للمستخدمين بتخصيص سلوك,9.3,"[بالإشارة, الخطط, للسماح, للمستخدمين, بتخصيص, ...","[شرة, خطط, سمح, خدم, خصص, سلك]","[إشارة, خطط, سماح, مستخدم, تخصيص, سلوك]"
102,بناءً على قيمهم الخاصة. هناك قلق مشابه بين ال...,بناءً قيمهم الخاصة قلق مشابه المحللين الأمنيين...,8.9,"[بناءً, قيمهم, الخاصة, قلق, مشابه, المحللين, ا...","[بنء, قيم, خصة, قلق, شبه, حلل, امن, حكم, استبد...","[بناء, قيم, خاصة, قلق, مشابه, محلل, الأمنيين, ..."
103,ستقيد ضوابط التصدير لإدارة بايدن في أكتوبر 202...,ستقيد ضوابط التصدير لإدارة بايدن ، المدى القصي...,8.8,"[ستقيد, ضوابط, التصدير, لإدارة, بايدن, ،, المد...","[تقد, ضبط, صدر, إدر, بيد, ،, مدى, قصر, قل،, وص...","[قيد, ضوابط, تصدير, إدارة, بايدن, ،, مدى, قصير..."
104,المعهد الدولي للدراسات الاستراتيجية - IISS,المعهد الدولي للدراسات الاستراتيجية,9.5,"[المعهد, الدولي, للدراسات, الاستراتيجية]","[عهد, دول, درس, استراتيجية]","[معهد, دولي, دراسة, الاستراتيجية]"


# **RNN & BidirectionalRNN & GRU & LSM ⛓**

In [202]:
df['data'] = [' '.join(df['lemm Text'][i]) for i in range(len(df))]

X = df['data']
y = df['relevance_score']

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [221]:
tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>') # Consider top 5000 words
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)
X_test_seq = tokenizer.texts_to_sequences(X_test)

max_length = 100
X_train_pad = pad_sequences(X_train_seq, maxlen=max_length, padding='post', truncating='post')
X_val_pad = pad_sequences(X_val_seq, maxlen=max_length, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length, padding='post', truncating='post')
X_train_pad

array([[ 11,  20, 125, ...,   0,   0,   0],
       [  6,  27,  11, ...,   0,   0,   0],
       [  2,   3, 327, ...,   0,   0,   0],
       ...,
       [681, 227,  33, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [682, 308, 126, ...,   0,   0,   0]], dtype=int32)

In [219]:
# 1. Simple RNN
model_rnn = Sequential()
model_rnn.add(Embedding(5000, 128, input_length=max_length))
model_rnn.add(SimpleRNN(64))
model_rnn.add(Dense(1, activation='linear')) # Regression output

# 2. Bidirectional RNN
model_bi_rnn = Sequential()
model_bi_rnn.add(Embedding(5000, 128, input_length=max_length))
model_bi_rnn.add(Bidirectional(LSTM(64)))
model_bi_rnn.add(Dense(1, activation='linear'))

# 3. GRU
model_gru = Sequential()
model_gru.add(Embedding(5000, 128, input_length=max_length))
model_gru.add(GRU(64))
model_gru.add(Dense(1, activation='linear'))

# 4. LSTM
model_lstm = Sequential()
model_lstm.add(Embedding(5000, 128, input_length=max_length))
model_lstm.add(LSTM(64))
model_lstm.add(Dense(1, activation='linear'))


models = [model_rnn, model_bi_rnn, model_gru, model_lstm]
for model in models:
    print(f"{model.name} training ....")
    model.summary()
    model.compile(loss='mse', optimizer='adam', metrics=['mae'])
    model.fit(X_train_pad, y_train, epochs=20, validation_data=(X_val_pad, y_val))
    print(f"Training Done")

sequential_65 training ....
Model: "sequential_65"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_65 (Embedding)    (None, 500, 128)          640000    
                                                                 
 simple_rnn_18 (SimpleRNN)   (None, 64)                12352     
                                                                 
 dense_64 (Dense)            (None, 1)                 65        
                                                                 
Total params: 652417 (2.49 MB)
Trainable params: 652417 (2.49 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/20
3/3 [==============================] - 2s 345ms/step - loss: 73.2334 - mae: 8.2842 - val_loss: 67.3150 - val_mae: 7.9856
Epoch 2/20
3/3 [==============================] - 1s 208ms/step - loss: 63.7403 - mae: 7.6838 - val_loss: 59.1194 - val_m

# **EVALUATION ✈**


In [220]:
from sklearn.metrics import mean_squared_error, r2_score

for model in models:
    y_pred = model.predict(X_test_pad)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print("Model:", model.name)
    print("MSE:", mse)
    print("R-squared:", r2)

1/1 [==============================] - 0s 238ms/step
Model: sequential_65
MSE: 3.7077872081697936
R-squared: -0.05326681975107039
1/1 [==============================] - 1s 1s/step
Model: sequential_66
MSE: 3.2766034176609424
R-squared: 0.06921906044097115
1/1 [==============================] - 1s 521ms/step
Model: sequential_67
MSE: 3.7416009657836184
R-squared: -0.06287225473052982
1/1 [==============================] - 1s 581ms/step
Model: sequential_68
MSE: 3.557479171140585
R-squared: -0.010568989682519403
